In [1]:
#Libraries Used
import socket
import pickle

In [2]:
#Closing any previous socket connection
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s.close()

In [3]:
#Defining host and port
host = 'local host'
port = 5000
#Defining the Socket connection
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s.bind(('', port)) #Binding the port and socket
s.listen(5)


In [4]:
#Loading the public keys of all clients and PKDA
#Also, Loading the private key of PKDA
with open('publicKey_PKDA.pkl','rb') as file:
    publicKey_PKDA = pickle.load(file)
    
with open('privateKey_PKDA.pkl','rb') as file:
    privateKey_PKDA = pickle.load(file)
    
with open('publicKey_client1.pkl','rb') as file:
    publicKey_client1 = pickle.load(file)
    
with open('publicKey_client2.pkl','rb') as file:
    publicKey_client2 = pickle.load(file)
    
public_key_dict={}
public_key_dict['server']= publicKey_PKDA
public_key_dict['client1'] = publicKey_client1
public_key_dict['client2'] = publicKey_client2

In [5]:
print(publicKey_PKDA)

[103, 209]


In [6]:
def binary_modular_exponention(m,e,n):
    result = 1
    m = m % n
    if (m == 0):
        return 0
    while(e>0):
        if(e&1):
            result = (result*m) % n
        e = e>>1
        m = (m*m) % n
    return result

In [7]:
#Encryption Algorithm
def encrypt_data(encryption_key, message):
    cipher_text = ''
    for value in range(0, len(message)):
        encrypted_char = binary_modular_exponention(ord(message[value]), encryption_key[0], encryption_key[1])
        cipher_text += chr(encrypted_char)
    return cipher_text

In [8]:
#Decryption Algorithm
def decrypt_data(decryption_key, cipher_data):
    original_text = ''
    for value in range(0, len(cipher_data)):
        decrypted_char = binary_modular_exponention(ord(cipher_data[value]), decryption_key[0], decryption_key[1])
        original_text += chr(decrypted_char)
    return original_text

In [9]:
while(True):
    print('Waiting for connection')
    print()
    c, addr = s.accept() #Accepting Connection
    print("CONNECTION FROM:", str(addr))
    print()
    c.send(b"connected succcessfully to Public-Key Authority")
    
    #Accepting the Client Request - Request sent from Step1
    msg = c.recv(1024)
    print()
    received_request = msg.decode() #Received Request from Step - 1
    print('Received message from client : ', received_request)
    
    #Step 2
    #E(PRauth, [PUb || Request || T1])
    #Finding the public key of client requested to connect
    client_requested = received_request.split('||')[0]  
    print('Client whose public key is requested : ',client_requested)
    print()
    publicKey_client_requested = public_key_dict[client_requested]
    
    #Generating the response
    response = str(publicKey_client_requested[0]) + '||' + str(publicKey_client_requested[1]) +'||'+ received_request
    print('Response generated by server(decrypted form) : ',response)
    print()
    
    #Encrypting the Response
    encrypted_text = encrypt_data(privateKey_PKDA, response)
    print('Response sent by server(encrypted form) : ',encrypted_text)
    c.send(encrypted_text.encode())
    print()
    print('------------------Transaction end---------------------')
    c.close()

s.close()

Waiting for connection

CONNECTION FROM: ('127.0.0.1', 59670)


Received message from client :  client2||1635253162.6066768
Client whose public key is requested :  client2

Response generated by server(decrypted form) :  5||221||client2||1635253162.6066768

Response sent by server(encrypted form) :  F55kkÉ55,0¡¥4k55ÉiFkFiÉkT°8

------------------Transaction end---------------------
Waiting for connection

CONNECTION FROM: ('127.0.0.1', 59674)


Received message from client :  client1||1635253162.6205213
Client whose public key is requested :  client1

Response generated by server(decrypted form) :  5||161||client1||1635253162.6205213

Response sent by server(encrypted form) :  F55ÉÉ55,0¡¥4É55ÉiFkFiÉkTkFkÉi

------------------Transaction end---------------------
Waiting for connection



KeyboardInterrupt: 